# Retrieval‑Augmented Generation (RAG) Workshop & Notebook Guide  
*A step‑by‑step companion for developers new to GenAI*

## 0  |  Purpose of this Notebook

The goal of the workshop is to **contrast “plain prompting” with Retrieval‑Augmented Generation (RAG)** and let you experience, hands‑on, how adding a retrieval step and function‑calling tools makes GPT‑4o‑mini:

* **more accurate** (grounded answers, fewer hallucinations)  
* **faster & cheaper** on larger corpora  
* **deterministic** when you need it (e.g. function calls)

## 1  |  Getting Ready

| What | Why |
|------|-----|
| **Python ≥ 3.10** | required by LangChain 0.2+ |
| **OpenAI account & `OPENAI_API_KEY`** | to access `gpt‑4o‑mini` |
| **Packages** | `pip install -r requirements.txt` |
| **Workshop data** | 50 short markdown files (≈ 90 kB total) provided in `/data/docs/` |

Clone the repo or pull the zipped folder, then open **`RAG‑workshop.ipynb`** in Jupyter or VS Code.

In [ ]:
# Run tot install python packages.
%pip install -r requirements.txt

In [15]:
#Prompting API example.
from utils import gpt_4o_mini

query = "Say 'Hello world' in five different languages."
print(gpt_4o_mini(query))

Sure! Here’s "Hello world" in five different languages:

1. Spanish: ¡Hola, mundo!
2. French: Bonjour, le monde !
3. German: Hallo, Welt!
4. Italian: Ciao, mondo!
5. Japanese: こんにちは、世界！ (Konnichiwa, sekai!)


### 1.2. Baseline data, CEO addresses and interviews


In [1]:
import glob

# Index of file to be printed
file_index = 1

for filepath in glob.glob("data/docs/*.txt")[file_index:file_index+1]:
    with open(filepath, "r", encoding="utf-8") as f:
        print(f"Firm:{filepath} \nText:{f.read()} ")


Firm:data/docs\CGI.txt 
Text:1. Corporate Overview
1.1. ABOUT CGI

Founded in 1976 and headquartered in Montréal, Canada, CGI is a leading IT and business consulting services firm with
approximately 90,250 consultants and professionals worldwide. We use the power of technology to help clients accelerate
their holistic digital transformation.

CGI has a people-centered culture, operating where our clients live and work to build trusted relationships and to advance our
shared communities. Our consultants and professionals are committed to providing actionable insights that help clients
achieve their business outcomes. CGI’s global delivery centers complement our proximity-based teams, offering clients added
options that deliver scale, innovation and delivery excellence in every engagement.

End-to-end services and solutions

CGI delivers end-to-end services that help clients achieve the highest returns on their digital investments. We call this ROI-led
digitization. Our insights-driven e

In [2]:
# Token count
import importlib, utils
importlib.reload(utils)
from utils import count_tokens_in_docs

print(count_tokens_in_docs())

abn_amro.txt: 1734 tokens
CGI.txt: 2795 tokens
ing.txt: 1006 tokens
rabobank.txt: 1183 tokens

Total tokens across all files: 6718


## 2  |  Baselines — No RAG

### 2.1  Naïve “Stuff‑Everything” Prompt

Chunk all text together, question answered correctly at random.

In [3]:
import importlib, utils
importlib.reload(utils)
from utils import gpt_4o_mini
import glob

# Chunks all text files together in a single string.
text = ""
for filepath in glob.glob("data/docs/*.txt"):
    with open(filepath, "r", encoding="utf-8") as f:
        text += f.read()

SYSTEM = f"""The text between triple backtics are multiple CEO interviews from an annual report, I will ask you retrieval-questions about it: ```{text}```"""
query = "Are the individual interviews question and answer-based and in what year? Check every firm seperately and provide me with one answer per firm."
response = gpt_4o_mini(user_message=query,system_message=SYSTEM)
print(f"Response: {response.content}\n")

Response: 1. **ABN AMRO**: Yes, the interview is question and answer-based and it is from the year 2024.

2. **CGI**: No, the text does not contain an interview format; it provides a general overview and strategic insights instead. 

3. **ING**: Yes, the interview is question and answer-based and it is from the year 2024. 

4. **Rabobank**: Yes, the interview is question and answer-based and it is from the year 2024.



### 2.2  File looping, asking the question seperately per firm.

In [5]:
import glob
for filepath in glob.glob("data/docs/*.txt"):
    with open(filepath, "r", encoding="utf-8") as f:
        text = f.read()

    # Get the file name without the extension
    file_name = str(filepath).split("\\")[-1].split(".")[0].upper()

    # Query
    SYSTEM = f"""The text between triple backtics is a CEO interview from an annual report, I will ask you retrieval-questions about it: ```{text}```"""
    query = "Is this interview question and answer-based and in what year?"
    response = gpt_4o_mini(user_message=query,system_message=SYSTEM)
    print(f"File: {file_name} \nResponse: {response.content}\n")

File: ABN_AMRO 
Response: Yes, the text is structured as a question-and-answer interview. The interview pertains to the year 2024.

File: CGI 
Response: The text provided is not structured as a typical interview question and answer format; rather, it is a comprehensive overview of the company CGI, its services, vision, strategy, and competitive environment. The text outlines various aspects of CGI's operations and strategies but does not include specific questions or responses usually found in an interview. The year mentioned in the text references fiscal 2024 results, implying it relates to the period ending September 30, 2024.

File: ING 
Response: The text is not presented in a question-and-answer format; it is a letter addressed to stakeholders, specifically shareholders. It reflects on the company's performance and outlook for the future in the year 2024.

File: RABOBANK 
Response: The text you provided is not structured as a traditional question and answer interview but rather as

**Expected result:** Inaccurate answers for fully chunked text.

**Takeaway:** Reducing input information size improves answer accuracy. 

## 3  |  Function‑Calling Experiments

The next cell registers a simple calculator tool:

Example of many digit computation. A transformer does not 'reason', it infers the answer likely to be the case form a test-corpus.

### 3.1. Asking ChatGPT directly, incorrect!

In [144]:
import importlib, utils
importlib.reload(utils)
from utils import gpt_4o_mini

# Wrong answer:
print(gpt_4o_mini("What is 12534 x 37245?, return me only the answer, no seperators").content)

466128630


### 3.2. Using a tool call, infers arguments from the prompt.

In [175]:

import importlib, utils
importlib.reload(utils)
from utils import gpt_4o_mini, tool_schema
import json

#Define tool_schema for the function multiplier.
tool_schema = {
    "type": "function",
    "function": {
        "name": "multiplier",
        "parameters": {
            "type": "object",
            "description": "Multiply two numbers.",
            "properties": {
                "x": {
                    "type": "integer",
                    "description": "First number to multiply, must be integer or float.",
                },
                "y": {
                    "type": "integer",
                    "description": "First number to multiply, must be integer or float.",
                },
            },
            "required": ["x", "y"],
        },
    },
}

#Call arguments.
arguments = json.loads(gpt_4o_mini("What is 12534 x 37245?",tool_schema=tool_schema).tool_calls[0].function.arguments)

#Calculate answer using the arguments.
print(arguments["x"]*arguments["y"])


466828830


**Expected result:** Inaccurate computation when asking ChatGPT directly.

**Takeaway:** Questions with exact answers should be answered using a tool-call.

## 4  |  Intro to Embeddings

### 4.1  Word Embeddings & Cosine Similarity

In [181]:
from sentence_transformers import SentenceTransformer, util
for model in ["all-MiniLM-L6-v2"]:
    print(model)

    #Baseline similarity
    sim = util.cos_sim(SentenceTransformer(model).encode("cat"), SentenceTransformer(model).encode("kitten"))
    print(sim)

    #Decreasing attention for word that differs
    sim = util.cos_sim(SentenceTransformer(model).encode("cat falls from the sky"), SentenceTransformer(model).encode("kitten falls from the sky"))
    print(sim)


all-MiniLM-L6-v2
tensor([[0.7882]])
tensor([[0.8582]])


Expect similarity around **0.8 – 0.9** → words are semantically close.

### 4.2  Text Embeddings: Good vs Bad Models

| Model | Expected similarity (`"GraphQL tutorial"` ↔ `"Learn GraphQL in depth"`) |
|-------|---------------------------------------------|
| **`all-MiniLM-L6-v2`** | **≈ 0.80** (good) |
| **`...` CLS** | **≈ 0.20** (bad) |

Some “general” models optimise for *token prediction*, not semantic retrieval.

## 5  |  Building a Tiny Vector Store

In [ ]:
from langchain_community.vectorstores import FAISS
index = FAISS.from_documents(docs, model)
retriever = index.as_retriever(search_kwargs={"k": 4})

*Tip:* Keep the index serialized to disk so you can reuse it between sessions.

## 6  |  LangChain RAG Pipeline (Working Demo)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model="gpt-4o-mini"),
    retriever=retriever,
    return_source_documents=True
)
qa_chain.invoke({"query": query})

*Outcome:* GPT first fetches the top‑K relevant chunks, then answers with citations.

## 7  |  Scaling Experiments

### 7.1  “Brute‑Force” Paragraph Loop (Again)
Run against all 50 docs → ~30 s, 50 API calls.

### 7.2  RAG: Filter ➜ Generate

1. **Retrieve `k=4`** docs (~0.9 s)  
2. Loop only their paragraphs (~2 s total)

| Method | Latency | Tokens billed | Accuracy |
|--------|---------|---------------|----------|
| Brute loop | 30 s | 20 k | ✅ |
| RAG filter+loop | **2 s** | **5 k** | ✅ |

## 8  |  Key Take‑aways

* **Don’t over‑stuff** prompts – use retrieval to keep context focused.  
* **Function calling** makes GPT predictable for structured tasks.  
* **Good embeddings** are half the battle; benchmark before you build.  
* **LangChain** reduces boilerplate, but it’s still just *Python* – inspect the objects!

## 9  |  Suggested Exercises

1. Swap `gpt-4o-mini` for `gpt-4o-128k` and compare cost vs latency.  
2. Replace FAISS with **Weaviate** or **Chroma**.  
3. Implement a **streamlit** UI that uses the LangChain chain.  
4. Try a **hybrid search** (BM25 + embeddings).

## 10  |  Further Reading

* OpenAI docs – *RAG best practices*  
* LangChain docs – *RetrievalQA*  
* “Improving Faithfulness in LLM‑generated Answers” (white‑paper, 2024)

---

Happy hacking — and enjoy watching your old‑school prompts evolve into robust RAG pipelines!